In [1]:
import h5py
import numpy as np
from data_loader import *
from numpy.fft import *

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import figure


import keras
from keras.models import Model, load_model
from keras.utils import plot_model, multi_gpu_model

import keras.backend as K

### Keras Conv2D
from keras.layers import Input, Reshape, MaxPooling2D, ZeroPadding2D, Flatten, Dropout, Dense, LSTM,Activation,add
# from keras.layers import Conv2D


from convolutional import Complex_Conv1D


/home/morads/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# path = "/datax/yzhang/army_challenge/training_data/"
path = '/datax/yzhang/deepsig/2018.01/GOLD_XYZ_OSC.0001_1024.hdf5'
f = h5py.File(path, 'r')

classes = ['32PSK',                                                                     
 '16APSK',
 '32QAM',
 'FM',
 'GMSK',
 '32APSK',
 'OQPSK',
 '8ASK',
 'BPSK',
 '8PSK',
 'AM-SSB-SC',
 '4ASK',
 '16PSK',
 '64APSK',
 '128QAM',
 '128APSK',
 'AM-DSB-SC',
 'AM-SSB-WC',
 '64QAM',
 'QPSK',
 '256QAM',
 'AM-DSB-WC',
 'OOK',
 '16QAM']


In [18]:
data = np.asarray(f["X"])
labels = np.asarray(f["Y"])
snr = np.asarray(f["Z"])
print(data.shape)

(2555904, 1024, 2)


In [19]:
# if set to true, we will add noise to the data
# and train a model to classify snr 0: SNR<0; 1: SNR>=0; 2: NOISE
train_snr = False

new_labels = []

def snr_to_one_hot(snr, labels):
    num_unique_labels = 3
    
    new_labels = np.zeros((labels.shape[0], num_unique_labels))    
    
    new_labels[np.where(snr<0)[0],0] = 1
    new_labels[np.where(snr>=0)[0],1] = 1
    
    return new_labels
    
    
data = np.transpose(data, (0,2,1))
    
if train_snr:
    labels = snr_to_one_hot(snr, labels)
    
print(data.shape)
print(labels.shape)
print(snr.shape)

(2555904, 2, 1024)
(2555904, 24)
(2555904, 1)


In [20]:
# load noise data from army
if train_snr:
    blc = 2
    path = "/datax/yzhang/army_challenge/training_data/"
    if blc == 0:
        path = "/datax/yzhang/training_data/"

    army_noise_data = []

    num_noise_data = labels.shape[0] // 2
    for i in range(15):
        data_file = path + "training_data_chunk_" + str(i) + ".pkl"
        army_noise_data.append(LoadModRecData(data_file, 1., 0., 0., load_mods=["NOISE"], verbose=False).signalData)

    army_noise_data = np.concatenate(army_noise_data)

    print(army_noise_data.shape)    

In [21]:
# make random gaussian noise
if train_snr:
    gaussian_noise = np.random.standard_normal(army_noise_data.shape)
    std = np.random.uniform(-2,2,gaussian_noise.shape[0])[:,np.newaxis]
    gaussian_noise = std[:,np.newaxis]*gaussian_noise

    noise_data = np.concatenate((army_noise_data, gaussian_noise))
    noise_labels = np.zeros((noise_data.shape[0],3))
    noise_labels[:,2] = 1

    noise_snr = -100*np.ones((noise_data.shape[0],1))
    print("noise", noise_data.shape)

In [22]:
# add noise data to training set
if train_snr:
    data = np.concatenate((data, noise_data))
    labels = np.concatenate((labels, noise_labels))
    snr = np.concatenate((snr, noise_snr))

In [23]:
print(data.shape)
print(labels.shape)
print(snr.shape)

(2555904, 2, 1024)
(2555904, 24)
(2555904, 1)


In [24]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues, labels=[]):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [25]:
### Real Model

# def inception(input_img, height = 1, fs=[64,64,64,64,64], with_residual=False, tw_tower=False):
#     tower_1 = Conv2D(filters=fs[0], kernel_size=[height, 1], padding='same', activation='relu')(input_img)
#     tower_2 = Conv2D(filters=fs[2], kernel_size=[height, 1], padding='same', activation='relu')(input_img)
#     tower_2 = Conv2D(filters=fs[3], kernel_size=[height, 9], padding='same', activation='relu')(tower_2)
#     tower_3 = Conv2D(filters=fs[2], kernel_size=[height, 1], padding='same', activation='relu')(input_img)
#     tower_3 = Conv2D(filters=fs[3], kernel_size=[height, 4], padding='same', activation='relu')(tower_3)
#     if tw_tower:
#         tower_5 = Conv2D(filters=fs[2], kernel_size=[height, 1], padding='same', activation='relu')(input_img)
#         tower_5 = Conv2D(filters=fs[3], kernel_size=[height, 12], padding='same', activation='relu')(tower_5)
#     tower_4 = MaxPooling2D(3, strides=1, padding='same')(input_img)
#     tower_4 = Conv2D(filters=fs[4], kernel_size=1, padding='same', activation='relu')(tower_4)
#     if tw_tower:
#         output = keras.layers.concatenate([tower_1, tower_2, tower_3, tower_4, tower_5], axis = 3)
#     else:
#         output = keras.layers.concatenate([tower_1, tower_2, tower_3, tower_4], axis = 3)
#     if with_residual and output.shape==input_img.shape:
#         output = output+input_img
#     print()
#     return output

# def out_tower(x, num_classes, dr=0.5):
#     x = Dropout(dr)(x)
#     output = Flatten()(x)
#     out    = Dense(num_classes, activation='softmax')(output)
#     return out

# def googleNet(x, data_format='channels_last', num_classes=24,num_layers=[1,2,3,2], features=[1,1,1,1,1]):
#     x = Reshape(in_shp + (1,), input_shape=in_shp)(x)
#     x = Conv2D(filters=64*features[0], kernel_size=[2,7], strides=[2,2], data_format=data_format, padding='same', activation='relu')(x)
#     x = MaxPooling2D([1, 3], strides=[1,2], padding='same')(x)
#     for dep in range(num_layers[0]):
#         y = x
#         x = Conv2D(filters=192*features[1], kernel_size=[1, 3], strides=[1,1], padding='same', activation='relu')(x)
#         if dep > 0:
#             x = add([x,y])
#     x = MaxPooling2D([1,3], strides=[1,2], padding='same')(x)
#     for dep in range(num_layers[1]):
#         y = x
#         x = inception(x, height=2, fs=np.array([64,64,64,64,64])*features[2], tw_tower=True)
#         if dep > 0:
#             x = add([x,y])
#     x = MaxPooling2D([1,3], strides=2, padding='same')(x)
#     for dep in range(num_layers[2]):
#         y = x
#         x = inception(x, height=2, fs=np.array([32,32,32,32,32])*features[3], with_residual=True)
#         if dep > 0:
#             x = add([x,y])

#     x = Conv2D(filters=128, kernel_size=1, padding='same', activation='relu')(x)
#     out = out_tower(x, num_classes, dr=0.5)
#     #out = Average()([out_mid, out_late])
#     return out


# in_shp = (2, 1024)
# input_img = Input(shape=in_shp)
# out = googleNet(input_img,data_format='channels_last', num_classes=labels[0].shape[0])
# model = Model(inputs=input_img, outputs=out)

# model.summary()

In [26]:
### Complex Model
def inception(input_img, fs=[64,64,64,64,64], with_residual=False, tw_tower=False):
    tower_1 = Complex_Conv1D(filters=fs[0], kernel_size=[1,1], padding='same', activation='relu')(input_img)
    tower_2 = Complex_Conv1D(filters=fs[2], kernel_size=[1,1], padding='same', activation='relu')(input_img)
    tower_2 = Complex_Conv1D(filters=fs[3], kernel_size=[1,9], padding='same', activation='relu')(tower_2)
    tower_3 = Complex_Conv1D(filters=fs[2], kernel_size=[1,1], padding='same', activation='relu')(input_img)
    tower_3 = Complex_Conv1D(filters=fs[3], kernel_size=[1,4], padding='same', activation='relu')(tower_3)
    if tw_tower:
        tower_5 = Complex_Conv1D(filters=fs[2], kernel_size=[1, 1], padding='same', activation='relu')(input_img)
        tower_5 = Complex_Conv1D(filters=fs[3], kernel_size=[1, 12], padding='same', activation='relu')(tower_5)
    tower_4 = MaxPooling2D(3, strides=1, padding='same')(input_img)
    tower_4 = Complex_Conv1D(filters=fs[4], kernel_size=[1, 1], padding='same', activation='relu')(tower_4)
    if tw_tower:
        output = keras.layers.concatenate([tower_1, tower_2, tower_3, tower_4, tower_5], axis = 3)
    else:
        output = keras.layers.concatenate([tower_1, tower_2, tower_3, tower_4], axis = 3)
    if with_residual and output.shape==input_img.shape:
        output = output+input_img
    return output

def out_tower(x, num_classes, dr=0.5):
    x = Dropout(dr)(x)
    output = Flatten()(x)
    out    = Dense(num_classes, activation='softmax')(output)
    return out

def googleNet(x, data_format='channels_last', num_classes=24,num_layers=[1,1,2,1], features=[1,1,1,1,1]):
    x = Reshape(in_shp + (1,), input_shape=in_shp)(x)
    x = Complex_Conv1D(filters=64*features[0], kernel_size=[1,6], strides=[1,2], data_format=data_format, padding='same', activation='relu')(x)
    x = MaxPooling2D([1, 3], strides=[1,2], padding='same')(x)
    for dep in range(num_layers[0]):
        y = x
        x = Complex_Conv1D(filters=128*features[1], kernel_size=[1,3], strides=[1,1], padding='same', activation='relu')(x)
        if dep > 0:
            x = add([x,y])
    x = MaxPooling2D([1,3], strides=[1,2], padding='same')(x)
    for dep in range(num_layers[1]):
        y = x
        x = inception(x, fs=np.array([32,32,32,32,32])*features[2], tw_tower=True)
        if dep > 0:
            x = add([x,y])
    x = MaxPooling2D([1,3], strides=[1,2], padding='same')(x)
    for dep in range(num_layers[2]):
        y = x
        x = inception(x, fs=np.array([32,32,32,32,32])*features[3], with_residual=True)
        if dep > 0:
            x = add([x,y])

    x = Complex_Conv1D(filters=64, kernel_size=[1,1], padding='same', activation='relu')(x)
    out = out_tower(x, num_classes, dr=0.5)
    #out = Average()([out_mid, out_late])
    return out


in_shp = (2, 1024)
input_img = Input(shape=in_shp)
out = googleNet(input_img,data_format='channels_last', num_classes=24)
model = Model(inputs=input_img, outputs=out)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 2, 1024)      0                                            
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 2, 1024, 1)   0           input_2[0][0]                    
__________________________________________________________________________________________________
complex__conv1d_24 (Complex_Con (None, 2, 512, 64)   896         reshape_2[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_7 (MaxPooling2D)  (None, 2, 256, 64)   0           complex__conv1d_24[0][0]         
__________________________________________________________________________________________________
complex__c

In [27]:
# # Uncomment to visualize architecture
# plot_model(model, to_file='model.png', show_shapes = True)

# figure(figsize = (15,20))
# img = mpimg.imread('model.png')
# plt.imshow(img, aspect='auto')
# plt.show()

In [28]:
def shuffle_data(data,labels, snr=None):
    idx = np.random.permutation(labels.shape[0])
    if not snr is None:
        return data[idx], labels[idx], snr[idx]
    return data[idx], labels[idx]

def split(data,labels, snr, train, val):
    l = data.shape[0]
    train_id = l * train // 100
    val_id = train_id + l * val // 100
         
    return  data[:train_id], labels[:train_id], snr[:train_id],                     \
            data[train_id:val_id], labels[train_id:val_id], snr[train_id:val_id],   \
            data[val_id:], labels[val_id:], snr[val_id:] 

# def normalize(sig):
#     print(sig.shape)
        
def batch_generator(sig_data, sig_labels, batch_size=512, noise=None, shuffle=False, freq_shift=np.pi, normalize=True):
    while True:
        data = np.copy(sig_data)
        labels = np.copy(sig_labels)
        N = data.shape[0]

        if shuffle:
            idx = np.random.permutation(data.shape[0])
            data, labels = shuffle_data(data, labels)
            
        if normalize:
            for i in range(data.shape[0]):
                sig = data[i][0] + 1j * data[i][1]
                sig = sig/np.std(sig)

                data[i][0] = sig.real
                data[i][1] = sig.imag

            
        steps = N // batch_size
        
        for i in range(steps):
            beg = i*batch_size
            end = beg+batch_size
            
            batch_data, batch_labels = sig_data[beg:end], sig_labels[beg:end]
            
            if noise:
                x,y,z = batch_data.shape
                batch_data += noise * np.random.randn(x, y, z)
                
            if freq_shift:
                shift = np.random.uniform(-freq_shift,freq_shift,batch_data.shape[0])
                carrier = np.exp(1j*shift[:,np.newaxis] * np.arange(1024).reshape((1,1024)))
                c_r = carrier.real; c_i = carrier.imag
                batch_data[:,0,:], batch_data[:,1,:] = batch_data[:,0,:]*c_r - batch_data[:,1,:]*c_i, \
                                                       batch_data[:,0,:]*c_i + batch_data[:,1,:]*c_r


            yield batch_data, batch_labels
        
        if noise:
            data = np.copy(sig_data)     

In [29]:
data, labels, snr = shuffle_data(data, labels, snr)

train_data, train_labels, train_snr, \
val_data, val_labels, val_snr, \
test_data, test_labels, test_snr = split(data, labels, snr, 80, 10)

In [30]:
print(data.shape)

(2555904, 2, 1024)


In [31]:
train_batch_size = 512
val_batch_size = 512

train_generator = batch_generator(train_data, train_labels, batch_size=512, noise=None, shuffle=True, freq_shift=None)
val_generator = batch_generator(val_data, val_labels, batch_size=512, noise=None, shuffle=False, freq_shift=None)

tsteps = train_data.shape[0] // train_batch_size
vsteps = val_data.shape[0] // val_batch_size

# print(next(train_generator)[0].shape)

In [32]:
print("train: ", train_data.shape, train_labels.shape, train_snr.shape)
print("val: ", val_data.shape, val_labels.shape, val_snr.shape)
print("test: ", test_data.shape, test_labels.shape, test_snr.shape)

train:  (2044723, 2, 1024) (2044723, 24) (2044723, 1)
val:  (255590, 2, 1024) (255590, 24) (255590, 1)
test:  (255591, 2, 1024) (255591, 24) (255591, 1)


In [ ]:
# model = multi_gpu_model(model, gpus=2)
number_of_epochs = 10
model.compile(loss='categorical_crossentropy', optimizer='adam')
filepath = '/tmp/morads/convmodrecnets_CNN2_0.5.wts.h5'

try:
    history = model.fit_generator(train_generator,
        nb_epoch=number_of_epochs,
        steps_per_epoch=tsteps,
        verbose=1,
        validation_data=val_generator,
        validation_steps=vsteps,
         callbacks = [
              keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
              keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
         ]) 
except(StopIteration):
    pass
    
model.save('small_complex2.h5')  

print("Done")

/home/morads/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  from ipykernel import kernelapp as app
/home/morads/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=3993, verbose=1, validation_data=<generator..., validation_steps=499, callbacks=[<keras.ca..., epochs=10)`
  from ipykernel import kernelapp as app


Epoch 1/10


In [ ]:
def getSNRindices(snrLabels, snr):
    return np.where(snrLabels==snr)[0]

def getModIndices(labels, mod):
    labl = np.argmax(labels, axis=1)
    return np.where(labl==mod)[0]
    

In [ ]:
filepath = '/tmp/morads/convmodrecnets_CNN2_0.5.wts.h5'
model = load_model(filepath)
# model = load_model("mod_classifier24.h5")

In [ ]:
# just accuracies
decimate = 5
eval_data, eval_snr, eval_labels = np.copy(test_data[::decimate]), np.copy(test_snr[::decimate]), np.copy(test_labels[::decimate])


for i in range(eval_data.shape[0]):
    eval_data[i] /= np.std(eval_data[i][0] + 1j*eval_data[i][1])
        
        
for SNR in np.unique(snr):
    snrIndices = getSNRindices(eval_snr, SNR)
    signalData = eval_data[snrIndices]
    
    
    
#     shift = np.random.uniform(-np.pi/8,np.pi/8,signalData.shape[0])
#     carrier = np.exp(1j*shift[:,np.newaxis] * np.arange(1024).reshape((1,1024)))
#     c_r = carrier.real; c_i = carrier.imag
    

#     temp = signalData.copy()
    
#     signalData[:,0,:] = temp[:,0,:]*c_r - temp[:,1,:]*c_i
#     signalData[:,1,:] = temp[:,0,:]*c_i + temp[:,1,:]*c_r

    
    signalLabels = np.argmax(eval_labels[snrIndices], axis = 1)
    preds = np.argmax(model.predict(signalData), axis = 1)
    correct = 0

    for i in range(preds.shape[0]):
        if preds[i] == signalLabels[i]:
            correct+=1
    
    print("SNR: {}, Accuracy: {}, n: {}".format(SNR, correct/preds.shape[0], signalData.shape[0]))
    
    

In [ ]:
# model = load_model("snr_predict3classes_freq_shifted.h5")

decimate = 100

eval_data, eval_snr, eval_labels = np.copy(test_data[::decimate]), np.copy(test_snr[::decimate]), np.copy(test_labels[::decimate])

for SNR in [10]:
    snrIndices = getSNRindices(eval_snr, SNR)
    signalData = eval_data[snrIndices]
    
    shift = np.random.uniform(-np.pi,np.pi,signalData.shape[0])
    carrier = np.exp(1j*shift[:,np.newaxis] * np.arange(1024).reshape((1,1024)))
    c_r = carrier.real; c_i = carrier.imag
    

    temp = signalData.copy()
    
    f = plt.figure(figsize=(8,8))
    index = 3
    plt.plot(np.abs(fftshift(fft(signalData[index][0].copy() + 1j *signalData[index][1].copy() ))))
#     plt.plot(fftshift(fft(signalData[index][0].copy() + 1j *signalData[index][1].copy() )))
    plt.show()
    
    signalData[:,0,:] = temp[:,0,:]*c_r - temp[:,1,:]*c_i
    signalData[:,1,:] = temp[:,0,:]*c_i + temp[:,1,:]*c_r
    
    f = plt.figure(figsize=(8,8))
    index = 3
    print(shift[index])
    plt.plot(np.abs(fftshift(fft(signalData[index][0].copy() + 1j *signalData[index][1].copy() ))))

#     plt.plot(fftshift(fft(signalData[index][0].copy() + 1j *signalData[index][1].copy() )).real)
#     plt.plot(fftshift(fft(signalData[index][0].copy() + 1j *signalData[index][1].copy() )).imag)
    plt.show()
    
    
    signalLabels = np.argmax(eval_labels[snrIndices], axis = 1)
    preds = np.argmax(model.predict(signalData), axis = 1)
    correct = 0

    for i in range(preds.shape[0]):
        if preds[i] == signalLabels[i]:
            correct+=1
    
    print("SNR: {}, Accuracy: {}, n: {}".format(SNR, correct/preds.shape[0], signalData.shape[0]))
    
    

In [ ]:
filepath = '/tmp/morads/convmodrecnets_CNN2_0.5.wts.h5'
filepath = 'mod_classifier21.h5'
model = load_model(filepath)
decimate = 5
eval_data, eval_snr, eval_labels = np.copy(test_data[::decimate]), np.copy(test_snr[::decimate]), np.copy(test_labels[::decimate])


In [ ]:
# for SNR in np.arange(-20,31,8):
#     snrIndices = getSNRindices(snr, SNR)
    
#     signalData = data[snrIndices]
#     signalLabels = np.argmax(labels[snrIndices], axis = 1)
#     preds = np.argmax(model.predict(signalData), axis = 1)
    
#     correct = 0
    
#     conf = np.zeros((len(classes),len(classes)))
#     confnorm = np.zeros((len(classes),len(classes)))

#     for i in range(preds.shape[0]):
#         if preds[i] == signalLabels[i]:
#             correct+=1
#         conf[signalLabels[i], preds[i]] += 1
    
#     for i in range(0,len(classes)):
#         confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
        
#     plt.figure(figsize=(10,10))
#     plot_confusion_matrix(confnorm, labels=classes, title="SNR: {} ConvNet Confusion Matrix".format(SNR))
#     plt.show()

        
#     print("SNR: {}, Accuracy: {}".format(SNR, correct/preds.shape[0]))
    
    



# print(eval_labels.shape)

# print(model.predict(eval_data[:2]).shape)

# for i in range(24):
#     idx = getModIndices(eval_labels, i)
    
#     mod_data = eval_data[idx]
#     mod_labels = eval_labels[idx]
#     mod_snr = eval_snr[idx]
    
#     snr = 10
    
#     idx = getSNRindices(mod_snr, snr)
#     mod_data = mod_data[idx]
#     mod_labels = mod_labels[idx]
#     mod_snr = mod_snr[idx]
    
    
#     plt.figure(figsize=(8,8))
#     plt.title(classes[i])
    
#     f = mod_data[0][0] + 1j * mod_data[0][1]
#     f = fftshift(fft(f))
#     plt.plot(f.real)
#     plt.plot(f.imag)
    
#     plt.show()
    
    
# print(eval_labels.shape)    
# # 
for SNR in np.unique(snr):
    snrIndices = getSNRindices(eval_snr, SNR)
    signalData = eval_data[snrIndices]
    

    signalLabels = np.argmax(eval_labels[snrIndices], axis = 1)
    preds = np.argmax(model.predict(signalData), axis = 1)
    correct = 0

    for i in range(preds.shape[0]):
        if preds[i] == signalLabels[i]:
            correct+=1
    
    print("SNR: {}, Accuracy: {}, n: {}".format(SNR, correct/preds.shape[0], signalData.shape[0]))
      
        
    
